# 영화 리뷰 데이터로 감성 예측하기

In [1]:
#warning 메시지 표시 안함
import schedule
import time
import warnings
warnings.filterwarnings(action = 'ignore')
import pandas as pd
import re
import numpy as np
import pymysql
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#사전 로딩 데이터(약 4분 소요)

# 훈련용 데이터인 ratings_train.txt 파일을 주피터 노트북에서 로드
nsmc_train_df = pd.read_csv('./train.txt', encoding = 'utf8', sep = ',', engine = 'python', names=['document','label'])
nsmc_train_df

# 훈련용 데이터셋의 정보를 확인
nsmc_train_df.info()

# 결측치 제거하기
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]

# 수정된 nsmc_train_df의 정보를 다시 확인
nsmc_train_df.info()

# 감성 분류 클래스의 구성을 확인
nsmc_train_df['label'].value_counts()

# 평가용 데이터인 ratings_test.txt 파일을 로드
nsmc_test_df = pd.read_csv('ratings_test.txt', encoding = 'utf8', sep = '\t', engine = 'python')
nsmc_test_df.head()
nsmc_test_df.info()

#document 칼럼이 Null인 샘플 제거
nsmc_test_df = nsmc_test_df[nsmc_test_df['document'].notnull()] 

nsmc_test_df['label'].value_counts()

nsmc_test_df['document'] = nsmc_test_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
nsmc_test_df.head()

# 분석모델 구축
# 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt 객체를 생성
from konlpy.tag import Okt
okt = Okt()

# 문장을 토큰화하기 위해 okt_tokenizer 함수를 정의하고 
# okt.morphs() 함수를 사용하여 형태소 단위로 토큰화 작업을 수행
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

#사이킷런의 TfidfVectorizer를 이용하여 TF-IDF 벡터화에 사용할 tfidf 객체를 생성
from sklearn.feature_extraction.text import TfidfVectorizer

# 토큰의 단어 크기ngram_range는 1~2개 단어로 함
# 토큰은 출현 빈도가 최소min_df 1번 이상이고 최대 max_df 90% 이하인 것만 사용
tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range = (1, 2), min_df = 1, max_df = 0.9)

# 벡터화할 데이터nsmc_train_df['document']에 대해 벡터 모델 tfidf의 내부 설정값을 조정fit( )
tfidf.fit(nsmc_train_df['document'])

# 벡터로 변환을 수행
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

#벡터로 변환된 분석모델 확인
print(nsmc_train_tfidf[:10])

#감성 분류 모델 구축
# 사이킷런의 LogisticRegression 클래스에 대해 객체 SA_lr을 생성
from sklearn.linear_model import LogisticRegression
SA_lr = LogisticRegression(random_state = 0)

# nsmc_train_tfidf를 독립변수 X로 하고 label 컬럼을 종속 변수 Y로 하여 
# 로지스틱 회귀 모델SA_lr의 내부 설정값을 조정fit( )
SA_lr.fit(nsmc_train_tfidf, nsmc_train_df['label'])

from sklearn.model_selection import GridSearchCV

#하이퍼 매개변수 C에 대해 비교 검사를 할 6개 값[1, 3, 3.5, 4, 4.5, 5]을 params로 하고,
params = {'C': [1, 3, 3.5, 4, 4.5, 5]}

# 교차 검증cv을 3, 모형 비교 기준은 정확도로 설정, scoring='accuracy'하여 GridSearchCV 객체를 생성
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1)

# GridSearchCV 객체에 nsmc_train_tfidf와 label 컬럼에 대해 설정값을 조정fit( ) 
SA_lr_grid_cv.fit(nsmc_train_tfidf, nsmc_train_df['label'])

# GridSearchCV에 의해 찾은 최적의 C 매개변수 best_params와 최고 점수 best_score를 출력하여 확인
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

##최적 매개변수의 best 모델 저장
SA_lr_best = SA_lr_grid_cv.best_estimator_

#분석모델 평가
#평가용 데이터의 피처 벡터화
# 평가용 데이터 nsmc_test_df['document']에 tfidf 객체를 적용하여 벡터 변환을 수행 transform( )
nsmc_test_tfidf = tfidf.transform(nsmc_test_df['document'])

#감성 분류 모델SA_lr_best에 nsmc_test_tfidf 벡터를 사용하여 감성을 예측 predict( )
test_predict = SA_lr_best.predict(nsmc_test_tfidf)

# 평가용 데이터의 감성 결과값 nsmc_test_df['label']과 
# 감성 예측값test_predict을 기반으로 정확도를 계산 accuracy_score( )하여 출력
from sklearn.metrics import accuracy_score
print('감성 분석 정확도 : ', round(accuracy_score(nsmc_test_df['label'], test_predict), 3))


#코멘트 테이블에 감석분석결과 업뎃하는 메서드
def emotional(data):
    conn=pymysql.connect(
        user='root',
        passwd='1234',
        host='localhost',
        db='travel',
        charset='utf8')
    cursor=conn.cursor()
    sql = 'update comment set emotion = %s where cid=%s'
    cursor.execute(sql, data)
    conn.commit()
    conn.close()


#크롤링 준비
path = 'C:/devtools/pythonwork/Lesson/chromedriver'

# 구글 사이트 접근 환경 설정 완료
driver = webdriver.Chrome(path)
action = ActionChains(driver)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31448 entries, 0 to 31447
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  31448 non-null  object
 1   label     31448 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 491.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31448 entries, 0 to 31447
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  31448 non-null  object
 1   label     31448 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 737.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21506 entries, 0 to 21505
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        21506 non-null  int64 
 1   document  21504 non-null  object
 2   label     21506 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 504.2+ KB
  (0, 213483)	0.1783705070

# 새로운 텍스트로 감성 예측 확인하기
1. 감성 분류 모델에 새로운 텍스트를 직접 입력하여 감성 예측을 수행

In [4]:
def do_emotion():
    # 관광지 번호만큼 반복문 실행
    for i in range(10):

        num = i+1
        url = 'http://localhost:8080/busan/' + str(num)

        # 검색한 관광지의 리뷰 화면 호출
        driver.implicitly_wait(2)
        driver.get(url)
        driver.set_window_size(700, 900)
        driver.implicitly_wait(1)
        #리뷰 갯수 가져오기
        #/html/body/div/div[4]/h6
        review_count = driver.find_elements_by_id('/html/body/div/div[4]/h6/span[2]')
        # 리뷰 갯수만큼 리뷰리스트 담기
        driver.implicitly_wait(2)
        review_count = driver.find_elements_by_xpath('/html/body/div/div[4]/h6/span[2]')
        total_review=0;
        #리뷰갯수 담을 객체 생성
        for i in review_count:
    #        print(i.text)
            total_review = int(i.text)
            #리뷰 갯수만큼 for문
            for j in range(total_review):
                k=j+1
                review=''
                cid=0
                driver.implicitly_wait(2)
                review_crawling = driver.find_elements_by_xpath('/html/body/div/div[4]/table/tbody/tr['+str(k)+']/td[3]')
                cid_crawling = driver.find_elements_by_xpath('/html/body/div/div[4]/table/tbody/tr['+str(k)+']/td[1]')
                for l in review_crawling:
                    driver.implicitly_wait(2)
                    review_crawling = driver.find_elements_by_xpath('/html/body/div/div[4]/table/tbody/tr['+str(k)+']/td[3]')
                    review = str(l.text)
                for m in cid_crawling:               
                    cid_crawling = driver.find_elements_by_xpath('/html/body/div/div[4]/table/tbody/tr['+str(k)+']/td[1]')
                    cid = int(m.text)
    #                print(cid)
    #                print(review)
                    data=(cid, cid)
                    emotional(data)

                    #0) 크롤링 텍스트에 대한 전처리 수행
                    review = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(review) ; print(review)

                    # 전처리 후 배열에 담긴 문자가 없을 경우(ex: 영어 숫자 이모티콘만 존재) : 0(중립감성)
                    if (len(review)==0):
                        data = (0,cid )
                        emotional(data)
                    else :
                        review = [" ".join(review)] ; print(review)
                        #1) 입력 텍스트의 피처 벡터화
                        st_tfidf = tfidf.transform(review)
                        st_tfidf
                        #2) 최적 감성 분석 모델에 적용하여 감성 분석 평가
                        st_predict = SA_lr_best.predict(st_tfidf)
                        st_predict

                        #1) 입력 텍스트의 피처 벡터화
                        st_tfidf = tfidf.transform(review)
                        #print(st_tfidf)

                        #2) 최적 감성 분석 모델에 적용하여 감성 분석 평가
                        st_predict = SA_lr_best.predict(st_tfidf)
                        #print(st_predict)

                        if(st_predict == 0):
                            # 부정 : emotion=1
                            data = (1,cid )
                            emotional(data)
                        else :
                            # 긍정 : emotion=2
                            data = (2,cid )
                            emotional(data)
do_emotion()

['요리보고']
['요리보고']
[]
[]
['댓글', '작성']
['댓글 작성']
['댓글', '시간출력']
['댓글 시간출력']
['시간이']
['시간이']
['검색', '완료된', '관광지의', '해시태그', '뿌려주기']
['검색 완료된 관광지의 해시태그 뿌려주기']
['좋아요']
['좋아요']
['이뻐요']
['이뻐요']
['아름다워요']
['아름다워요']
['사진', '찍기', '좋습니다']
['사진 찍기 좋습니다']
['들어가라']
['들어가라']
['댓글', '테스트']
['댓글 테스트']
['좋아요']
['좋아요']
['싫어요']
['싫어요']
['이뻐요']
['이뻐요']
['별로에여']
['별로에여']
['너무', '좋아']
['너무 좋아']
['너무', '싫어']
['너무 싫어']
['아이', '좋아']
['아이 좋아']
['아이', '싫어']
['아이 싫어']
['별로에여']
['별로에여']
['좋아']
['좋아']
['별로에여']
['별로에여']
['좋아']
['좋아']
['별로에여']
['별로에여']
['야경', '너무', '이뻐요']
['야경 너무 이뻐요']
['사람', '많아서', '불편했어요']
['사람 많아서 불편했어요']
['어린왕자', '동상이랑', '사진찍기', '좋아요', '음식이', '너무', '비싸요']
['어린왕자 동상이랑 사진찍기 좋아요 음식이 너무 비싸요']
['야경', '너무', '이뻐요']
['야경 너무 이뻐요']
['별로에요']
['별로에요']
['바다', '너무', '시원해서', '좋았어요']
['바다 너무 시원해서 좋았어요']
[]


In [ ]:
# 8시간마다 do()실행 (seconds, minutes, hours, days, weeks)
schedule.every(8).hours.do(do_emotion)
while True:
    schedule.run_pending()

['한국전쟁', '피란민들의', '애환이', '서려', '있는', '장소입니다', '지금은', '부산', '원도심의', '추억도', '어려', '있는', '곳이기도', '하며', '부산', '지역', '문화', '예술계에서', '일하는', '분들이', '모여', '또따또가라는', '문화예술지역으로', '개발하는', '곳이기도', '하지요', '계단', '문화관을', '포함하여', '지역', '골목골목을', '천천히', '둘러보면', '아기자기한', '재미가', '있습니다']
['한국전쟁 피란민들의 애환이 서려 있는 장소입니다 지금은 부산 원도심의 추억도 어려 있는 곳이기도 하며 부산 지역 문화 예술계에서 일하는 분들이 모여 또따또가라는 문화예술지역으로 개발하는 곳이기도 하지요 계단 문화관을 포함하여 지역 골목골목을 천천히 둘러보면 아기자기한 재미가 있습니다']
['별로', '지루', '아쉽다']
['별로 지루 아쉽다']
[]
[]
[]
[]
[]
['해변에서', '바라보는', '광안대교', '야경이', '정말', '예뻣어요']
['해변에서 바라보는 광안대교 야경이 정말 예뻣어요']
['하루종일', '바라보고', '있어도', '질리지', '않을', '풍경']
['하루종일 바라보고 있어도 질리지 않을 풍경']
[]
['ㅈㄷ', '겨ㅑㅅ퍄ㅕㅁㄷ', 'ㅛ소ㅓ']
['ㅈㄷ 겨ㅑㅅ퍄ㅕㅁㄷ ㅛ소ㅓ']
['ㅇㄱ', 'ㅑㅌ교어ㅕㅇ뇨ㅓㅏㅌㄹ화넌교ㅑㅌ셔아ㅓ오ㅓㅇ서셔ㅏ셔', 'ㅐㅇ', 'ㅐㅑ쇼ㅓㄴ쇼ㅓㅏㅇ셔ㅏㅇ셔ㅏㅣ야ㅣㅇㄹ']
['ㅇㄱ ㅑㅌ교어ㅕㅇ뇨ㅓㅏㅌㄹ화넌교ㅑㅌ셔아ㅓ오ㅓㅇ서셔ㅏ셔 ㅐㅇ ㅐㅑ쇼ㅓㄴ쇼ㅓㅏㅇ셔ㅏㅇ셔ㅏㅣ야ㅣㅇㄹ']
['ㄷ', 'ㅕ얼퇕호']
['ㄷ ㅕ얼퇕호']
[]
[]
[]
['ㅀㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ']
['ㅀㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ']
['ㅕ요ㅕㅇ롱로ㅓㄹㅇ홓로롤홀홓롷ㄹㄹㄹㄴ', 'ㅛㄴ굔', 'ㅕㄴ거뇨ㅏㄴ셔ㅏㅇ셔ㅣㅏ